# Empowering Healthcare with Symbolic Learning and Knowledge Graph Embeddings

### Overview: Welcome!!! Today we will play with Symbolic learning and KGE models like TransH or RotatE over the Lung Cancer KG. In Lung Cancer KG, a patient is described by medical characteristics such as smoking habit, cancer stage, mutation type, age, gender and occurence of relapse etc. Here, the task is to predict a patient with recommended drug or relapse condition.

#### Install prerequistes and import necessary modules

In [ ]:
!git clone https://github.com/SDM-TIB/SymbolicLearning_KGE.git

In [ ]:
%%capture
!pip install -r /content/SymbolicLearning_KGE/requirements.txt

#### Symbolic Learning Execution

In [ ]:
%cd /content/SymbolicLearning_KGE/SymbolicLearning
!python symbolic_predictions.py

#### KGE Models Execution

In [ ]:
!python /content/SymbolicLearning_KGE/KGEmbedding/kge.py --dataset_path "/content/SymbolicLearning_KGE/KG/OriginalKG/LungCancer.tsv" --output_dir "/content/SymbolicLearning_KGE/KGEmbedding/OriginalKG" --results_path "/content/SymbolicLearning_KGE/KGEmbedding/OriginalKG/" --models TransH

#### Perform Link Prediction (to predict the missing link, i.e., tail or head entity)

In [ ]:
!python /content/SymbolicLearning_KGE/KGEmbedding/link_prediction.py --results "/content/SymbolicLearning_KGE/KGEmbedding/OriginalKG/" --model_name "TransH" --head "3561_Patient" --relation "hasRelapse_Progression"

In [ ]:
import pandas as pd
import json
pred_result_path = "/content/SymbolicLearning_KGE/KGEmbedding/OriginalKG/TransH/prediction_result.csv"
pred = pd.read_csv(pred_result_path)

# Display the prediction result, i.e., top-5
pred.head(5)

#### Impact of Symbolic Learning over KGE Model Evaluation

In [ ]:
def generate_dataframe(model_name, file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    hits_at_10 = data['metrics']['tail']['realistic']['hits_at_10']
    mrr = data['metrics']['tail']['realistic']['inverse_harmonic_mean_rank']

    df = pd.DataFrame({
        'Benchmark': [kg_name],
        'Hits@10': [hits_at_10],
        'MRR': [mrr]
    })
    return df

model_files = [
    ('OriginalKG', '/content/SymbolicLearning_KGE/KGEmbedding/OriginalKG/TransH/results.json'),
    ('EnrichedKG', '/content/SymbolicLearning_KGE/KGEmbedding/EnrichedKG/TransH/results.json'),
    ('TransformedKG', '/content/SymbolicLearning_KGE/KGEmbedding/TransformedKG/TransH/results.json')
]

dfs = []
for kg_name, file_path in model_files:
    df = generate_dataframe(kg_name, file_path)
    dfs.append(df)

# Concatenate all DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)
print(final_df)